In [ ]:
%matplotlib inline
import numpy as np
import cv2
    
import matplotlib.pyplot as plt

import sys
sys.path.append("../")

import tensorflow as tf 
assert int(tf.__version__[:1]) < 2.0, "해당 코드는 1.x에서만 동작합니다."
tf.logging.set_verbosity(tf.logging.ERROR)

# \[ MNIST 데이터셋 - CRNN \]

MNIST 데이터셋을 통해 정상적으로 동작하는지를 확인해보도록 함

## 1. 데이터 가져오기 

In [ ]:
from utils.dataset import SerializationDataset

train_set = SerializationDataset('mnist','train',
                                 digit=5,pad_range=(3,10))
validation_set = SerializationDataset('mnist','validation',
                                      digit=5,pad_range=(3,10))
test_set = SerializationDataset('mnist','test',
                                digit=(3,8),pad_range=(3,10))

## 2. 데이터 Generator 가져오기

In [ ]:
from models.generator import DataGenerator

train_gen = DataGenerator(train_set, 
                          batch_size=32)
valid_gen = DataGenerator(validation_set, 
                          batch_size=100, 
                          shuffle=False)
test_gen = DataGenerator(test_set, 
                         batch_size=500, 
                         shuffle=False)

## 3. 모델 구성하기

In [ ]:
from models.layers import ConvFeatureExtractor, Map2Sequence 
from models.layers import BLSTMEncoder, CTCDecoder
from models.losses import ctc_loss

import tensorflow as tf
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense

In [ ]:
height = 28
num_classes = 10
n_hidden = 16
n_lstm = 256
K.clear_session()

# For Gray Scale Image & Dynamic width
inputs = Input(shape=(height, None, 1),name='image')

# CRNN Model
conv_maps = ConvFeatureExtractor(n_hidden=n_hidden,
                                 name='feature_extractor')(inputs)
feature_seqs = Map2Sequence(name='map_to_sequence')(conv_maps)
lstm_seqs = BLSTMEncoder(n_units=n_lstm)(feature_seqs)

# 우리의 출력 형태는 class 수에 Blank Label을 하나 더해 #classes + 1 만큼을 출력
output_seqs = Dense(num_classes+1,
                    activation='softmax',
                    name='output_seqs')(lstm_seqs)

In [ ]:
from models.optimizer import AdamW

# 모델 구성하기
# (1) 학습 모델 구성하기
y_true =  tf.placeholder(shape=(None,None), dtype=tf.int32)
trainer = Model(inputs, output_seqs, name='trainer')
trainer.compile('adam',
                loss={"output_seqs":ctc_loss},
                target_tensors=[y_true])

### caution

`K.ctc_batch_cost`에 이용되는 Input Tensor의 Interface는 아래와 같습니다.

* y_true: tensor `(samples, max_string_length)` containing the truth labels.
* y_pred: tensor `(samples, time_steps, num_categories)` containing the prediction, or output of the softmax.

In [ ]:
# (2) 예측 모델 구성하기
predictions = CTCDecoder(beam_width=100)(output_seqs)
predictor = Model(inputs, predictions[0], name='predictor')

## 4. 모델  학습시키기


In [ ]:
trainer.fit_generator(train_gen,
                      epochs=10,
                      validation_data=valid_gen)

## 5. 모델 평가하기

In [ ]:
for image, true_label in zip(*test_gen[0]):
    result = predictor.predict(image[np.newaxis])
    predict_seq = "".join([str(char) for char in result.ravel()])
    plt.title(f'label : {predict_seq}')
    plt.imshow(image[:,:,0])
    plt.show()